# Embedding Model Demonstration

This notebook demonstrates the core concept behind Retrieval Augmented Generation - **Vector Embeddings**.


In [1]:
%load_ext dotenv
%dotenv
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
import plotly.express as px
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from src.util.env_check import get_embed_model

Loading the Embedding Model

In [2]:
embedding_model = get_embed_model()
print(f"Loaded embedding model: {embedding_model}")

Loaded embedding model: model='qwen3-embedding:0.6b' base_url='http://localhost:11434' client_kwargs={} mirostat=None mirostat_eta=None mirostat_tau=None num_ctx=None num_gpu=None num_thread=None repeat_last_n=None repeat_penalty=None temperature=None stop=None tfs_z=None top_k=None top_p=None


### Simple Sentence Demonstration

Embedding models are machine learning models used to capture meaning behind data. They are often based on transformer architectures now but they started off as simple shallow neural networks. They convert complex unstructured data (text, images, or audio) into dense numerical vectors (embeddings). These vectors capture semantic meaning, allowing computers to identify relationships, context, and similarities, essential for applications like semantic search, recommendation systems, and Retrieval-Augmented Generation (RAG)

To demonstrate embedding models on a small example lets embed 2 sets of sentences:
1.  Animals/Nature
2.  Computer Science/Data Mining

We expect the model to group semantically similar sentences together.

In [3]:
sentences = [
    #Animals
    "The quick brown fox jumps over the dog.",
    "The neigbourhood dog barks all night.",
    "My cat likes to chase dogs in the yard.",
    "Cheetas are very fast.",
    
    #Data Mining
    "Data mining involves discovering patterns in large datasets.",
    "Neural networks are a subset of machine learning algorithms.",
    "Clustering algorithms group similar data points together.",
    "Principal component analysis reduces the dimensionality of data."
]


# For color coding
labels = ["Animal","Animal","Animal","Animal","Tech","Tech","Tech","Tech"]

# Generate embeddings
embeddings = embedding_model.embed_documents(sentences)
embeddings_array = np.array(embeddings)

print(f"Generated embeddings shape: {embeddings_array.shape}")

Generated embeddings shape: (8, 1024)


In [4]:
embeddings_array[0]

array([ 0.02527663, -0.04816985, -0.00368367, ...,  0.03945439,
       -0.00913975, -0.00020009])

### Visualization (PCA)
Embeddings are high-dimensional vectors. Lets use Principal Component Analysis (PCA) to reduce them to 3 dimensions so we can plot them.

You can interact with the plot.

In [5]:
simple_pca = PCA(n_components=3)
reduced_embeddings = simple_pca.fit_transform(embeddings_array)

df = pd.DataFrame({
    'x': reduced_embeddings[:, 0],
    'y': reduced_embeddings[:, 1],
    'z': reduced_embeddings[:, 2],
    'sentence': sentences,
    'category': labels
})

fig = px.scatter_3d(
    df, 
    x='x', y='y', z='z',
    color='category',
    hover_data={'x': False, 'y': False, 'z': False, 'sentence': True},
    title="3D Sentence Embeddings (interactive plot)",
    color_discrete_map={'Animal': 'green', 'Tech': 'blue'}
)

fig.update_traces(marker=dict(size=5))
fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))

fig.show()

### Textbook Analysis

Now we apply this to the Textbook:
1.  Load the Textbook.
2.  Use the `contents.json` map to split it into chapters.
3.  Embed a sample of text chunks from each chapter.
4.  Visualize if chapters cluster separately in vector space.

In [6]:
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

json_path = "../data/processed/contents.json"
pdf_path = "../data/raw/Textbook.pdf"

with open(json_path, "r") as f:
    chapters_json = json.load(f)

loader = PyPDFLoader(pdf_path)
pages = loader.load()

print(f"Loaded {len(pages)} pages from the textbook.")

Loaded 746 pages from the textbook.


### Processing Chapters
Using the logic from `advanced_ingest.py` to split the book by chapters.

In [7]:
PAGE_OFFSET = 26 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chapter_chunks = []
chapter_labels = []

# Processing only the first 6 chapters for clarity in the plot
target_chapters = chapters_json[:6]

for chapter in target_chapters:
    start_page = chapter["start_page"]
    end_page = chapter["end_page"]
    ch_title = chapter["title"]
    ch_num = chapter["chapter_number"]
    
    start_idx = start_page + PAGE_OFFSET - 1
    if end_page is not None:
        end_idx = end_page + PAGE_OFFSET - 1
        current_pages = pages[start_idx : end_idx + 1]
    else:
        current_pages = pages[start_idx : ]
        
    full_text = "\n".join([p.page_content for p in current_pages])
    
    doc = Document(page_content=full_text, metadata={"chapter": f"Ch {ch_num}"})
    chunks = text_splitter.split_documents([doc])
    
    # Take some chunks out of each chapter
    sample_chunks = chunks[:15]
    
    for chunk in sample_chunks:
        contextualized_text = f"Chapter {ch_num}: {ch_title}\n{chunk.page_content}"
        chapter_chunks.append(contextualized_text)
        chapter_labels.append(f"Ch {ch_num}")

print(f"Total chunks prepared for embedding: {len(chapter_chunks)}")

Total chunks prepared for embedding: 90


Generate Chapter Embeddings

In [8]:
book_embeddings = embedding_model.embed_documents(chapter_chunks)
book_embeddings_array = np.array(book_embeddings)

### 3D Visualization of chapters
We plot the chunks in 3D space, coloring them by chapter. We should see chunks from the same chapters cluster together.

In [9]:
pca = PCA(n_components=3)
reduced_book = pca.fit_transform(book_embeddings_array)

df = pd.DataFrame({
    'x': reduced_book[:, 0],
    'y': reduced_book[:, 1],
    'z': reduced_book[:, 2],
    'chapter': chapter_labels,
    'text': chapter_chunks
})

fig = px.scatter_3d(
    df, 
    x='x', y='y', z='z',
    color='chapter',
    hover_data={'x': False, 'y': False, 'z': False, 'chapter': True},
    title="Semantic Clusters of Textbook Chapters",
    color_discrete_sequence=px.colors.qualitative.Plotly
)

fig.update_traces(
    marker=dict(size=4, opacity=0.8, line=dict(width=0))
)

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=40),
    legend_title_text='Chapter',
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    )
)

fig.show()

The plot above demonstrates that semantically related text clusters together in vector space.

- Chunks from the same chapter appear close to each other.
- Chunks from different topics are separated.


### How This Applies to RAG (Retrieval Augmented Generation)

When a user asks a question, a RAG system performs the following steps:
1. Embed the query: It takes the users raw text question and passes it through the exact same embedding model used for the database.
2. Calculate similarity: It compares the querys vector against every chunks vector in the database to find the closest matches. 
3. Retrieve and generate: It takes the top K most similar chunks and feeds them to an LLM as context to formulate an accurate answer.

**Cosine Similarity**

To find the closest vectors, the most common metric used is cosine similarity. Instead of measuring the physical distance between two points, it measures the angle between two vectors. This makes it highly effective for text, as it focuses on the orientation (semantic meaning) rather than the magnitude (length of the document).

The formula for Cosine Similarity between vectors $A$ and $B$ is:
$$\text{Cosine Similarity} = \frac{A \cdot B}{\|A\| \|B\|}$$

A score of **1** means the vectors point in the exact same direction (highly similar), **0** means they are unrelated, and **-1** means they are completely opposite.

### Simple demonstration

In [10]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

user_query = "What wild cat is the quickest?"

query_embedding = embedding_model.embed_documents([user_query])[0]
query_vector = np.array(query_embedding)
query_vector



array([ 0.00352722,  0.00419386, -0.00748914, ...,  0.04388433,
       -0.01641025, -0.02402433])

This is essentially what happens when we run similarity search on a vectorstore

In [11]:

similarities = []
for sentence_vector in embeddings_array:
    sim = cosine_similarity(query_vector, sentence_vector)
    similarities.append(sim)

best_match_idx = np.argmax(similarities)
best_score = similarities[best_match_idx]
best_sentence = sentences[best_match_idx]

print(f"User Query: '{user_query}'\n")
print("-" * 50)
print(f"Top Database Match (Similarity Score: {best_score:.4f}):")
print(f"'{best_sentence}'")
print("-" * 50)

print("\nAll scores ranked:")
ranked_indices = np.argsort(similarities)[::-1]
for idx in ranked_indices:
    print(f"[{similarities[idx]:.4f}] {sentences[idx]}")

User Query: 'What wild cat is the quickest?'

--------------------------------------------------
Top Database Match (Similarity Score: 0.6797):
'Cheetas are very fast.'
--------------------------------------------------

All scores ranked:
[0.6797] Cheetas are very fast.
[0.5362] My cat likes to chase dogs in the yard.
[0.5324] The quick brown fox jumps over the dog.
[0.3996] The neigbourhood dog barks all night.
[0.2979] Principal component analysis reduces the dimensionality of data.
[0.2929] Data mining involves discovering patterns in large datasets.
[0.2901] Clustering algorithms group similar data points together.
[0.2507] Neural networks are a subset of machine learning algorithms.


Visualizing this similarity and how it applies to RAG

In [16]:
reduced_query = simple_pca.transform([query_vector])

all_reduced_embeddings = np.vstack([reduced_embeddings, reduced_query])
all_sentences = sentences + [user_query]
all_labels = labels + ["User Query"]

sizes = [5] * len(sentences) + [10]

df_query = pd.DataFrame({
    'x': all_reduced_embeddings[:, 0],
    'y': all_reduced_embeddings[:, 1],
    'z': all_reduced_embeddings[:, 2],
    'sentence': all_sentences,
    'category': all_labels,
    'size': sizes
})

fig = px.scatter_3d(
    df_query, 
    x='x', y='y', z='z',
    color='category',
    size='size',
    size_max=10,
    hover_data={'x': False, 'y': False, 'z': False, 'sentence': True, 'size': False},
    title="Visualizing the user query in vector space",
    color_discrete_map={'Animal': 'green', 'Tech': 'blue', 'User Query': 'red'}
)


fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))

fig.show()

In [17]:
user_query_rag = "What is Lp norm?"

query_embedding_rag = embedding_model.embed_documents([user_query_rag])[0]
query_vector_rag = np.array(query_embedding_rag)

reduced_query_rag = pca.transform([query_vector_rag])

all_points = np.vstack([reduced_book, reduced_query_rag])
all_texts = chapter_chunks + [user_query_rag]
all_categories = chapter_labels + ["USER QUERY"]
all_sizes = [5] * len(chapter_chunks) + [15]

df_rag = pd.DataFrame({
    'PC1': all_points[:, 0],
    'PC2': all_points[:, 1],
    'PC3': all_points[:, 2],
    'Text': all_texts,
    'Category': all_categories,
    'Size': all_sizes
})

rag_fig = px.scatter_3d(
    df_rag,
    x='PC1', y='PC2', z='PC3',
    color='Category',
    size='Size',
    size_max=15,
    hover_data={'PC1': False, 'PC2': False, 'PC3': False, 'Text': False, 'Category': True, 'Size': False},
    title="User question and book chapters",
    color_discrete_sequence=px.colors.qualitative.Plotly
)
rag_fig.update_layout(margin=dict(l=0, r=0, b=0, t=40))
rag_fig.show()

scores = [cosine_similarity(query_vector_rag, chunk_vector) for chunk_vector in book_embeddings_array]

top_indices = np.argsort(scores)[-3:][::-1]

context = ""
print(f"User Query: '{user_query_rag}'")
print("\n--- Top 3 Most Similar Textbook Chunks (Retrieved Context) ---")
for i, idx in enumerate(top_indices):
    context += chapter_chunks[idx]
    context += "\n"
    print(f"\n[Match {i+1}] Similarity Score: {scores[idx]:.4f}")
    print(f"Chapter: {chapter_labels[idx]}")
    print(f"Snippet: {chapter_chunks[idx][:300]}...")

User Query: 'What is Lp norm?'

--- Top 3 Most Similar Textbook Chunks (Retrieved Context) ---

[Match 1] Similarity Score: 0.5758
Chapter: Ch 3
Snippet: Chapter 3: Similarity and Distances
3.2 Multidimensional Data
Although multidimensional data are the simplest form of data, there is signiﬁcant diversity
indistancefunctiondesignacrossdiﬀerentattributetypessuchascategoricalorquantitative
data. This section will therefore study each of these types se...

[Match 2] Similarity Score: 0.5435
Chapter: Ch 3
Snippet: Chapter 3: Similarity and Distances
3.2. MULTIDIMENSIONAL DATA 67
Figure 3.2: Impact ofp on contrast
subset selection during preprocessing, because the relevance of features islocallydetermined
by the pair of objects that are being considered. Globally, all features may be relevant.
Whenmanyfeatures...

[Match 3] Similarity Score: 0.4818
Chapter: Ch 3
Snippet: Chapter 3: Similarity and Distances
particular application. For example, for a credit-scoring application, an attribute 

As we can see our question about Lp norms show up in the vector space closest to chunks from Chapter 3: Similarity and Distances. 

We pass these <u>retrieved</u> chunks as additional context to an LLM <u>augmenting</u> its limited knowledge while <u>generating</u> an answer.

In [ ]:
from src.util.env_check import get_llm_model
from langchain_core.messages import SystemMessage, HumanMessage

llm = get_llm_model()

system_message = (
    "You are an expert Data Mining Tutor helping a student study from Charu C. Aggarwal's 'Data Mining: The Textbook'.\n\n"
    
    "Your goal is to provide clear, educational responses structured into two distinct parts: \n"
    "1. Theoretical Explanation \n"
    "2. Python Code Implementation \n\n"
    
    "### Guidelines:\n"
    "- STRICT CONTEXT FOR THEORY: You MUST base your theoretical explanation ONLY on the provided context snippets. Do not invent theories, formulas, or include concepts not found in the text. "
    "If the context does not contain enough information to answer the question, state clearly: 'The provided text does not contain enough information to answer this.'\n"
    "- EXTERNAL KNOWLEDGE FOR CODE: Because the textbook focuses on mathematical theory, you are explicitly allowed and encouraged to use your general programming knowledge to write Python code (e.g., using pandas, numpy, scikit-learn). The code must accurately practically demonstrate the specific theoretical concepts discussed in the context.\n"
    "- MATH FORMATTING: When writing mathematical formulas, you MUST use LaTeX notation:\n"
    "  - Use double dollar signs for standalone equations (e.g., $$E=mc^2$$).\n"
    "  - Use single dollar signs for inline math (e.g., $x^2$).\n"
    "  - Do not use brackets like \\[ \\] or \\( \\) for math.\n"
    "- TONE AND STRUCTURE: Be encouraging, clear, and pedagogical. Use Markdown formatting, clear headings, and bullet points to make your explanations scannable and easy to digest.\n\n"
    
    "Here is the context:\n"
    f"{context}"
)

messages = [SystemMessage(content=system_message)]
    
messages.append(HumanMessage(content=user_query_rag))

response = llm.invoke(messages)

In [23]:
from IPython.display import display, Markdown
display(Markdown(response.content))

### Theoretical Explanation

The **Lp-norm** is a distance function commonly used to measure the difference between two data points in a multidimensional space. It is particularly useful for quantitative data. The Lp-norm between two data points \( X = (x_1, \ldots, x_d) \) and \( Y = (y_1, \ldots, y_d) \) is defined as:

$$
\text{Dist}(X, Y) = \left( \sum_{i=1}^{d} |x_i - y_i|^p \right)^{1/p}
$$

- **Parameters**:
  - The value of \( p \) determines the type of distance:
    - \( p = 2 \): The Euclidean distance, which is the straight-line distance between two points.
    - \( p = 1 \): The Manhattan distance, which is like the distance one would travel on a grid of streets (like those in Manhattan).
  - \( d \) is the dimensionality of the data.

- **Usage**:
  - The Lp-norm is versatile and can adapt to different types of data by varying \( p \). It is especially important in spatial applications where distances have clear physical interpretations.

### Python Code Implementation

Below is a Python implementation demonstrating how to calculate the Lp-norm between two points using the `numpy` library:

```python
import numpy as np

def lp_norm(X, Y, p=2):
    """
    Calculate the Lp-norm distance between two points X and Y.

    Parameters:
    X (array-like): The first data point.
    Y (array-like): The second data point.
    p (int): The order of the norm (default is 2, which is the Euclidean distance).

    Returns:
    float: The Lp-norm distance.
    """
    X = np.array(X)
    Y = np.array(Y)
    distance = np.sum(np.abs(X - Y) ** p) ** (1 / p)
    return distance

# Example usage
X = [1, 2, 3]
Y = [4, 5, 6]

# Calculate Euclidean distance (p=2)
euclidean_distance = lp_norm(X, Y, p=2)
print("Euclidean Distance:", euclidean_distance)

# Calculate Manhattan distance (p=1)
manhattan_distance = lp_norm(X, Y, p=1)
print("Manhattan Distance:", manhattan_distance)
```

This code defines a function `lp_norm` that calculates the Lp-norm distance between two vectors `X` and `Y`. You can specify the order `p` to switch between different norms (such as Euclidean and Manhattan).